In [1]:
# libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast
from sklearn.model_selection import train_test_split
import pickle

! pip install keras_tuner
import keras_tuner as kt


2025-01-31 18:51:37.331516: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#df1 = pd.read_csv("data/large_board_dataset.csv")
df2 = pd.read_csv("large_board_dataset2.csv")
df3 = pd.read_csv("large_board_dataset3.csv")
#df4 = pd.read_csv("data/large_board_dataset2-mikala.csv")
#df4 = pd.read_csv("data/player_one_moves.csv")
#df5 = pd.read_csv("data/player_two_moves.csv")


df = pd.concat([df2, df3], axis =0)


In [ ]:
df.head()
df["x"] = df["x"].apply(lambda x: np.array(ast.literal_eval(x)).reshape(2, 6, 7))

def preprocess_board(board):
    if board["whose_turn"] != "red":
        # Flip the layers to make each board look like it's from the perspective of "red", aka plus
        board["x"] = board["x"][::-1, :, :]
    return board

df = df.apply(preprocess_board, axis=1)

def flip_board(board, col):
    new_board = np.flip(board, axis = 2)
    # for now this needs to be done by index 2 because the input is still 2x6x7, but this can easily be updated
    new_column = 6 - col
    return new_board, new_column

flipped_x = []
flipped_y = []

for index, row in df.iterrows():
    board = row["x"]
    col = row["y"]
    new_board, new_column = flip_board(board, col)

    flipped_x.append(new_board)
    flipped_y.append(new_column)

new_df = pd.DataFrame({"x": flipped_x, "y": flipped_y})

df = pd.concat([df, new_df], ignore_index = True)

In [ ]:
x = df["x"]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 22)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = X_train.transpose(0, 2, 3, 1)  # Convert (num_samples, 2, 6, 7) to (num_samples, 6, 7, 2)
X_test = X_test.transpose(0, 2, 3, 1)

In [3]:
# TRYING WITH THE PICKLE FILE

df = pd.read_pickle("data/654914moves_6-7-2shape.pkl")
x = df["x"]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 22)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [4]:
def model_builder(hp):
        model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
                                kernel_size=(
                                    hp.Int('kernel_size_1_height', min_value=2, max_value=4, step=1),
                                    hp.Int('kernel_size_1_width', min_value=2, max_value=4, step=1)
                                    ),
                                activation=tf.nn.relu,
                                input_shape=(6, 7, 2)),
        tf.keras.layers.MaxPooling2D(pool_size =
                                    (hp.Int('maxpool_size_1_height', min_value=1, max_value=4, step=1),
                                    hp.Int('maxpool_size_1_width', min_value=1, max_value=4, step=1)
                                    ),strides=(1, 1), padding = "same"),

        tf.keras.layers.Conv2D(filters=hp.Int('filters_2', min_value=32, max_value=128, step=32),
                                kernel_size=(
                                    hp.Int('kernel_size_2_height', min_value=2, max_value=4, step=1),
                                    hp.Int('kernel_size_2_width', min_value=2, max_value=4, step=1)
                                    ),
                                activation=tf.nn.relu),

        tf.keras.layers.MaxPooling2D(pool_size =
                                    (hp.Int('maxpool_size_2_height', min_value=1, max_value=4, step=1),
                                    hp.Int('maxpool_size_2_width', min_value=1, max_value=4, step=1)
                                    ), strides = (1, 1), padding = "same"),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(hp.Int('dense_units_1', min_value = 32, max_value = 128, step = 32),
                                activation=tf.nn.relu,
                                kernel_regularizer = tf.keras.regularizers.l2(
                                        hp.Choice('l2_reg_dense1', values = [0.0001, 0.0001, 0.005, 0.01])
                                )),
        tf.keras.layers.Dropout(hp.Float('dropout_1', min_value = 0.1, max_value = 0.4, step = 0.1)),
        tf.keras.layers.Dense(hp.Int('dense_units_2', min_value = 32, max_value = 128, step = 32),activation=tf.nn.relu,
                                kernel_regularizer = tf.keras.regularizers.l2(
                                        hp.Choice('l2_reg_dense2', values = [0.0001, 0.0001, 0.005, 0.01])
                                )),
        tf.keras.layers.Dropout(hp.Float('dropout_2', min_value = 0.1, max_value = 0.4, step = 0.1)),
        tf.keras.layers.Dense(7,activation=tf.nn.softmax)
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(
                learning_rate = hp.Choice('learning_rate', values = [0.1, 0.001, 0.0001])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

        return model

In [5]:
# now we have a tunable model; set up kt.Hyperband
# hyperband paper: https://arxiv.org/pdf/1603.06560

tuner = kt.Hyperband(model_builder,
                    objective = 'val_accuracy',
                    max_epochs = 10,
                    factor = 3,
                    directory = 'cnn_tuning',
                    project_name = 'cnn_tuning_kt'
                    )


/Users/fmunting/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# add early stopping to reduce training time

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

# now run the search

tuner.search(X_train, y_train, epochs = 50, validation_split = 0.2, callbacks=[stop_early])

Trial 27 Complete [00h 12m 21s]
val_accuracy: 0.5088035464286804

Best val_accuracy So Far: 0.5816752314567566
Total elapsed time: 02h 16m 11s


In [7]:
# get the best hyperparameters out:

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best Filters (Conv1): {best_hps.get('filters_1')}")
print(f"Best Kernel Height (Conv1): {best_hps.get('kernel_size_1_height')}")
print(f"Best Kernel Width (Conv1): {best_hps.get('kernel_size_1_width')}")
print(f"Best Kernel Height (Conv2): {best_hps.get('kernel_size_2_height')}")
print(f"Best Kernel Width (Conv2): {best_hps.get('kernel_size_2_width')}")
print(f"Best Filters (Conv2): {best_hps.get('filters_2')}")
print(f"Best Dense Units (Layer 1): {best_hps.get('dense_units_1')}")
print(f"Best Dense Units (Layer 2): {best_hps.get('dense_units_2')}")
print(f"Best Dropout Rate (Layer 1): {best_hps.get('dropout_1')}")
print(f"Best Dropout Rate (Layer 2): {best_hps.get('dropout_2')}")
print(f"Best L2 Regularization (Layer 1): {best_hps.get('l2_reg_dense1')}")
print(f"Best L2 Regularization (Layer 2): {best_hps.get('l2_reg_dense2')}")
print(f"Best Learning Rate: {best_hps.get('learning_rate')}")
print(f"Best Maxpool Height (Conv1): {best_hps.get('maxpool_size_1_height')}")
print(f"Best Maxpool Width (Conv1): {best_hps.get('maxpool_size_1_width')}")
print(f"Best Maxpool Height (Conv2): {best_hps.get('maxpool_size_2_height')}")
print(f"Best Maxpool Width (Conv2): {best_hps.get('maxpool_size_2_width')}")


Best Filters (Conv1): 128
Best Kernel Height (Conv1): 4
Best Kernel Width (Conv1): 3
Best Kernel Height (Conv2): 3
Best Kernel Width (Conv2): 4
Best Filters (Conv2): 96
Best Dense Units (Layer 1): 32
Best Dense Units (Layer 2): 32
Best Dropout Rate (Layer 1): 0.2
Best Dropout Rate (Layer 2): 0.30000000000000004
Best L2 Regularization (Layer 1): 0.0001
Best L2 Regularization (Layer 2): 0.0001
Best Learning Rate: 0.001
Best Maxpool Height (Conv1): 2
Best Maxpool Width (Conv1): 1
Best Maxpool Height (Conv2): 3
Best Maxpool Width (Conv2): 1


In [8]:
# train the best model on the full dataset

best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    X_train, y_train,
    epochs=100,
    validation_split = 0.2
)

test_loss, test_acc = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# save the best model for future use
best_model.save('best_connect4_cnn.h5')

Epoch 1/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 70s 5ms/step - accuracy: 0.4158 - loss: 1.5257 - val_accuracy: 0.5239 - val_loss: 1.2183
Epoch 2/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 66s 5ms/step - accuracy: 0.5127 - loss: 1.2840 - val_accuracy: 0.5392 - val_loss: 1.1811
Epoch 3/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 68s 5ms/step - accuracy: 0.5327 - loss: 1.2353 - val_accuracy: 0.5476 - val_loss: 1.1509
Epoch 4/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - accuracy: 0.5437 - loss: 1.2056 - val_accuracy: 0.5617 - val_loss: 1.1243
Epoch 5/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 68s 5ms/step - accuracy: 0.5536 - loss: 1.1814 - val_accuracy: 0.5660 - val_loss: 1.1124
Epoch 6/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - accuracy: 0.5581 - loss: 1.1661 - val_accuracy: 0.5666 - val_loss: 1.1097
Epoch 7/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 69s 5ms/step - accuracy: 0.5635 - loss: 1.1501 - val_accuracy: 0.5718 - val_loss: 1.0949
Epoch 8/100
13099/13099 ━━━━━━━━━━━━━━━━━━━━ 68s 5ms/step - ac

In [9]:
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

4094/4094 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.5915 - loss: 1.0980
Test accuracy: 0.5926
